In [ ]:
from ARIMA_FUNCTIONS_REAL import *

# CAC 40

In [ ]:
# download data
dfCAC_ARIMA = get_data_ARIMA(index_symbol='^FCHI', start='2000-01-01')

### Hyperparameter Tunning SET-UP

In [ ]:
lookback_bars = 1000
validation_bars = 250
testing_bars = 250
lag = 21

p_max = 6
q_max = 6

SENS = 'SENS_0A'
PATH = F'CAC/Sensitivity Analysis/{SENS}'

TRANSACTION_COST = (0.2/100)

# ARIMA

In [ ]:
model_predictionsCAC, train_dataCAC, test_validation_dataCAC = main_arima(p_max, q_max, dfCAC_ARIMA, lookback_bars, validation_bars, testing_bars)

In [ ]:
df_testCAC_ARIMA = pd.concat(test_validation_dataCAC, axis=0)

In [ ]:
num = lookback_bars+validation_bars+lag

inv_y_test_LO = df_testCAC_ARIMA['Close'].values[validation_bars+lag:]
inv_y_pred_test_LO = model_predictionsCAC[validation_bars+lag:]

In [ ]:
df_inv_y_test_LO = pd.DataFrame(data={'Date': dfCAC_ARIMA.index[num:(len(inv_y_test_LO)+num)], 'inv_y__test': inv_y_test_LO})
df_inv_y_pred_test_LO = pd.DataFrame(data={'Date': dfCAC_ARIMA.index[num:(len(inv_y_test_LO)+num)], 'inv_y_pred_test': inv_y_pred_test_LO})

In [ ]:
fig_ARIMA(df_index=dfCAC_ARIMA, df_test=df_inv_y_test_LO, df_predictions=df_inv_y_pred_test_LO)

# EQUITY CURVE

# Long-Only

# Performance Metrics

In [ ]:
position_LO = np.where(df_inv_y_pred_test_LO['inv_y_pred_test'].shift(-1)>df_inv_y_test_LO['inv_y__test'],1,0)

In [ ]:
df_Equity_Curve = dfCAC_ARIMA[['Close']]

In [ ]:
df_Equity_Curve = df_Equity_Curve[num:len(inv_y_test_LO)+num]

In [ ]:
df_Equity_Curve['strat_return'] = df_Equity_Curve['Close'].pct_change().dropna()
df_Equity_Curve['bnh_return'] = df_Equity_Curve['Close'].pct_change().dropna()

In [ ]:
df_Equity_Curve['position'] = position_LO
df_Equity_Curve = transaction_cost(df_Equity_Curve, TRANSACTION_COST) 

In [ ]:
df_Equity_Curve["strategy"] = (df_Equity_Curve["strat_return"] * df_Equity_Curve['position'].shift(1))
df_Equity_Curve["strategy"] = (1+df_Equity_Curve["strategy"].fillna(0)).cumprod()

df_Equity_Curve['buy_n_hold'] = (1 + df_Equity_Curve['bnh_return'].fillna(0)).cumprod()

In [ ]:
fig_strategies(df_Equity_Curve)

In [ ]:
df_Equity_Curve['position'].value_counts()

# PERFORMANCE METRICS

In [ ]:
wyniki(np.array(df_Equity_Curve['buy_n_hold'].values), 'Equity_Curve_BuyAndHold')

In [ ]:
wyniki(np.array(df_Equity_Curve['strategy'].values), 'Equity_Curve_strategii')

In [ ]:
porownanie(np.array(df_Equity_Curve['strategy'].values), np.array(df_Equity_Curve['buy_n_hold'].values))

### TO CSV - SAVE

In [ ]:
STOCK_NAME_FOLDER = f'./Results/{PATH}/Visualisations'

if not os.path.exists(STOCK_NAME_FOLDER):
    os.makedirs(STOCK_NAME_FOLDER)

df_Equity_Curve.to_csv(f'{STOCK_NAME_FOLDER}/df_EC_LO_MAIN.csv')

# Long Short

In [ ]:
num = lookback_bars+validation_bars+lag

inv_y_test_LS = df_testCAC_ARIMA['Close'].values[validation_bars+lag:]
inv_y_pred_test_LS = model_predictionsCAC[validation_bars+lag:]

In [ ]:
df_inv_y_test_LS = pd.DataFrame(data={'Date': dfCAC_ARIMA.index[num:(len(inv_y_test_LS)+num)], 'inv_y__test': inv_y_test_LS})
df_inv_y_pred_test_LS = pd.DataFrame(data={'Date': dfCAC_ARIMA.index[num:(len(inv_y_test_LS)+num)], 'inv_y_pred_test': inv_y_pred_test_LS})

In [ ]:
fig_ARIMA(df_index=dfCAC_ARIMA, df_test=df_inv_y_test_LS, df_predictions=df_inv_y_pred_test_LS)

# Performance Metrics

In [ ]:
# position_LO = (test_model_predictions_LO>0.5).astype(int)
position_LS = np.where(df_inv_y_pred_test_LS['inv_y_pred_test'].shift(-1)>df_inv_y_test_LS['inv_y__test'],1,-1)

In [ ]:
df_Equity_Curve_LS = dfCAC_ARIMA[['Close']]

In [ ]:
df_Equity_Curve_LS = df_Equity_Curve_LS[num:len(inv_y_test_LS)+num]

In [ ]:
df_Equity_Curve_LS['strat_return'] = df_Equity_Curve_LS['Close'].pct_change().dropna()
df_Equity_Curve_LS['bnh_return'] = df_Equity_Curve_LS['Close'].pct_change().dropna()

In [ ]:
df_Equity_Curve_LS['position'] = position_LS
df_Equity_Curve_LS = transaction_cost(df_Equity_Curve_LS, TRANSACTION_COST) 

In [ ]:
df_Equity_Curve_LS["strategy"] = (df_Equity_Curve_LS["strat_return"] * df_Equity_Curve_LS['position'].shift(1))
df_Equity_Curve_LS["strategy"] = (1+df_Equity_Curve_LS["strategy"].fillna(0)).cumprod()

df_Equity_Curve_LS['buy_n_hold'] = (1 + df_Equity_Curve_LS['bnh_return'].fillna(0)).cumprod()

In [ ]:
fig_strategies(df_Equity_Curve_LS)

In [ ]:
df_Equity_Curve_LS['position'].value_counts()

# PERFORMANCE METRICS

In [ ]:
wyniki(np.array(df_Equity_Curve_LS['buy_n_hold'].values), 'Equity_Curve_BuyAndHold')

In [ ]:
wyniki(np.array(df_Equity_Curve_LS['strategy'].values), 'Equity_Curve_strategii')

In [ ]:
porownanie(np.array(df_Equity_Curve_LS['strategy'].values), np.array(df_Equity_Curve_LS['buy_n_hold'].values))

### TO CSV - SAVE

In [ ]:
STOCK_NAME_FOLDER = f'./Results/{PATH}/Visualisations'

if not os.path.exists(STOCK_NAME_FOLDER):
    os.makedirs(STOCK_NAME_FOLDER)

df_Equity_Curve_LS.to_csv(f'{STOCK_NAME_FOLDER}/df_EC_LS_MAIN.csv')